### Student Information
Name: 黃俊瑋 

Student ID: 108062308

GitHub ID: [tangerine1202](https://github.com/tangerine1202/DM2022-Lab2-Homework)

Kaggle name: [Chun Wei Huang](https://www.kaggle.com/tangerine1202)

Kaggle private scoreboard snapshot:

[Snapshot](img/pic0.png)

---

### Instructions

1. First: __This part is worth 30% of your grade.__ Do the **take home** exercises in the [DM2022-Lab2-master Repo](https://github.com/keziatamus/DM2022-Lab2-Master). You may need to copy some cells from the Lab notebook to this notebook. 


2. Second: __This part is worth 30% of your grade.__ Participate in the in-class [Kaggle Competition](https://www.kaggle.com/competitions/dm2022-isa5810-lab2-homework) regarding Emotion Recognition on Twitter by [this link](https://www.kaggle.com/t/2b0d14a829f340bc88d2660dc602d4bd). The scoring will be given according to your place in the Private Leaderboard ranking: 
    - **Bottom 40%**: Get 20% of the 30% available for this section.

    - **Top 41% - 100%**: Get (60-x)/6 + 20 points, where x is your ranking in the leaderboard (ie. If you rank 3rd your score will be (60-3)/6 + 20 = 29.5% out of 30%)   

    Submit your last submission __BEFORE the deadline (Nov. 22th 11:59 pm, Tuesday)_. Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the img folder of this repository and rerun the cell Student Information.

3. Third: __This part is worth 30% of your grade.__ A report of your work developping the model for the competition (You can use code and comment it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained. 


4. Fourth: __This part is worth 10% of your grade.__ It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook** and **add minimal comments where needed**.


Upload your files to your repository then submit the link to it on the corresponding e-learn assignment.

Make sure to commit and save your changes to your repository __BEFORE the deadline (Nov. 25th 11:59 pm, Friday)__.

## First Part: Take Home Exercises

- repo link: https://github.com/tangerine1202/DM2022-Lab2-Master

# Second Part: Kaggle Competition

In [1]:
from datetime import datetime 
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

from tqdm.notebook import tqdm

## Prepare Data

In [2]:
# ident = pd.read_csv('data/data_identification.csv', index_col='tweet_id')
# labels = pd.read_csv('data/emotion.csv', index_col='tweet_id')

# print('train size', ident[ident['identification'] == 'train'].shape)
# print('test  size', ident[ident['identification'] == 'test'].shape)

# print(ident.head(3))
# print(labels.head(3))

In [ ]:
# train_idxs = ident[ident['identification'] == 'train'].index
# train_labels = labels.loc[train_idxs, 'emotion']

# train_labels.head(3)

In [ ]:
# print('labels distribution')
# print(train_labels.value_counts())
# sns.barplot(x=train_labels.value_counts().index, y=train_labels.value_counts().values)

In [ ]:
# data = pd.read_json('data/tweets_DM.json', lines=True)

# # drop useless columns (only 1 unique value)
# data = data.drop(columns=['_index', '_type'])
# source = data['_source']

# # parsing tweet
# def parse_source(data):
#     data = data['tweet']
#     parsed_data = {}
#     for key, value in data.items():
#       parsed_data[key] = value
#     return parsed_data
# tweet = pd.DataFrame(source.map(parse_source).tolist())
# data = pd.concat([data, tweet], axis=1)
# data = data.drop(columns=['_source'])

# data.set_index('tweet_id', inplace=True)

# data.head(3)

In [ ]:
# train_data = data.loc[train_idxs, :].sort_values(by='tweet_id')
# test_data = data.loc[~data.index.isin(train_idxs), :].sort_values(by='tweet_id')
# train_labels = train_labels.sort_index()

# assert len(train_data) == len(train_labels)
# assert np.all(train_data.index == train_labels.index)
# print(len(data))
# print(len(train_data), len(test_data))

In [ ]:
# train_data.to_pickle('data/train_data.pkl')
# train_labels.to_pickle('data/train_labels.pkl')
# test_data.to_pickle('data/test_data.pkl')

# Custom Transferred model

In [2]:
from torch.utils.data import DataLoader
from torch.optim import AdamW

from datasets import Dataset, load_dataset
from datasets import load_metric
import evaluate

from transformers import AutoTokenizer
from transformers import AutoModel, AutoConfig, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import pipeline
from transformers import get_scheduler
from transformers import DataCollatorWithPadding
from transformers.modeling_outputs import TokenClassifierOutput

from transformers import XLNetTokenizer, XLNetForSequenceClassification

2022-11-16 01:25:08.116497: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-16 01:25:08.479333: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-16 01:25:09.354479: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/alan/.cuda/lib64:/home/alan/.cuda/extras/CUPTI/lib64:/home/alan/.cuda/lib64:/home/alan/.cuda/extras/CUPTI/lib64:
2022-11-16 01:25:09.354633: W tensorflow/stream_executor/platform/default/dso_loader.

In [3]:
# data
TRAIN_DATA_PATH = 'data/train_data.pkl'
TRAIN_LABEL_PATH = 'data/train_labels.pkl'
TEST_DATA_PATH = 'data/test_data.pkl'
CHECKPOINTS_PATH = 'checkpoints/'
EMOTION_NAMES = ['joy', 'anticipation', 'trust' , 'sadness' , 'disgust' , 'fear' , 'surprise', 'anger']
INPUT_COLUMNS = ['text', 'label', *[f'{emo}_ratio' for emo in EMOTION_NAMES]]

# constants
MODEL_NAME = 'cardiffnlp/twitter-roberta-base-sentiment-latest'
NUM_LABELS = 8
CHECKPOINTS_SIZE = 1000

# hyper-parameters
EPOCHS = 3
BATCH_SIZE = 32
LR = 1e-4
TRAIN_SIZE = 100000
EVAL_SIZE = 2000

In [4]:
emotion2id = {emotion: i for i, emotion in enumerate(EMOTION_NAMES)}
id2emotion = {i: emotion for i, emotion in enumerate(EMOTION_NAMES)}

In [5]:
df_train_X = pd.read_pickle(TRAIN_DATA_PATH)
df_train_y = pd.read_pickle(TRAIN_LABEL_PATH)
assert len(df_train_X) == len(df_train_y)
df_train = pd.concat([df_train_X, df_train_y], axis=1)

# numerical labels
df_train['label'] = df_train['emotion'].map(lambda x: emotion2id[x])

df_train.sample(5)

,_score,_crawldate,hashtags,text,joy_ratio,anticipation_ratio,trust_ratio,sadness_ratio,disgust_ratio,fear_ratio,surprise_ratio,anger_ratio,emotion,label
tweet_id,,,,,,,,,,,,,,
0x344305,239,2017-08-10 20:33:04,[SECP],Playing with national institutions is like pla...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,disgust,4
0x24a9c6,269,2016-08-24 02:10:48,"[BillOBrien, Texans, TNF]",Anyone consider that #BillOBrien may just be a...,0.247719,0.122807,0.108421,0.190877,0.236140,0.023860,0.030526,0.039649,disgust,4
0x1f9f26,205,2015-09-18 12:16:57,[],"My brother just said ""this kid I used to be fr...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,trust,2
0x1e9c2b,420,2016-10-22 14:23:13,"[Jesus, TheWalkingDead]","Damnit, #Jesus, stop being so...like Jesus! T...",0.053433,0.680370,0.021627,0.072114,0.102495,0.019505,0.037247,0.013209,anticipation,1
0x26bcdb,47,2015-01-24 02:07:17,[],Love changes lives. <LH>,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,joy,0


In [6]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, max_length=512)

In [7]:
# dataset = Dataset.from_pandas(df_train[INPUT_COLUMNS]])
# dataset

In [8]:
# tokenized_datasets = dataset.map(tokenize_function, batched=True)
# tokenized_datasets.save_to_disk(f'data/{MODEL_NAME}_tokenized_datasets')

In [9]:
tokenized_datasets = Dataset.load_from_disk(f'data/{MODEL_NAME}_tokenized_datasets')

In [10]:
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets = tokenized_datasets.remove_columns(["text", 'tweet_id'])
tokenized_datasets.set_format("torch")
tokenized_datasets

Dataset({
    features: ['labels', 'joy_ratio', 'anticipation_ratio', 'trust_ratio', 'sadness_ratio', 'disgust_ratio', 'fear_ratio', 'surprise_ratio', 'anger_ratio', 'input_ids', 'attention_mask'],
    num_rows: 1455563
})

In [11]:
small_train_dataset = tokenized_datasets.shuffle(seed=42).select(range(TRAIN_SIZE))
small_eval_dataset = tokenized_datasets.shuffle(seed=42).select(range(TRAIN_SIZE, TRAIN_SIZE + EVAL_SIZE))
train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=BATCH_SIZE, collate_fn=data_collator)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=BATCH_SIZE, collate_fn=data_collator)

print(f'training size: {len(small_train_dataset)}')
print(f'eval size: {len(small_eval_dataset)}')

Loading cached shuffled indices for dataset at data/cardiffnlp/twitter-roberta-base-sentiment-latest_tokenized_datasets/cache-6aa2f57568fda59c.arrow
Loading cached shuffled indices for dataset at data/cardiffnlp/twitter-roberta-base-sentiment-latest_tokenized_datasets/cache-6aa2f57568fda59c.arrow


training size: 100000
eval size: 2000


## Prepare model

In [12]:
# ref: https://towardsdatascience.com/adding-custom-layers-on-top-of-a-hugging-face-model-f1ccdfc257bd
class CustomModel(torch.nn.Module):
  def __init__(self, checkpoint, num_labels): 
    super(CustomModel,self).__init__() 
    self.num_labels = num_labels 

    #Load Model with given checkpoint and extract its body
    self.model = AutoModel.from_pretrained(checkpoint,config=AutoConfig.from_pretrained(checkpoint, output_attentions=True,output_hidden_states=True))
    self.seq_dropout = torch.nn.Dropout(0.3) 
    self.features_dropout = torch.nn.Dropout(0.0)
    self.classifier = torch.nn.Linear(768 + NUM_LABELS, num_labels) # load and initialize weights

  def forward(self, 
    input_ids=None, attention_mask=None, labels=None,
    joy_ratio=None, anticipation_ratio=None, trust_ratio=None, sadness_ratio=None, disgust_ratio=None, fear_ratio=None, surprise_ratio=None, anger_ratio=None
  ):
    # Extract outputs from the body
    # pretrained_outputs[0]=last hidden state
    pretrained_outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
    seq_outputs = self.seq_dropout(pretrained_outputs[0])[:, 0, :].view(-1, 768) # seq shape torch.Size([16, 768])

    # Add emotion ratios
    emo_ratio_outputs = torch.stack([joy_ratio, anticipation_ratio, trust_ratio, sadness_ratio, disgust_ratio, fear_ratio, surprise_ratio, anger_ratio], dim=1)
    features_outputs = self.features_dropout(emo_ratio_outputs) # emo_ratio shape torch.Size([16, 8])

    # Concatenate
    outputs = torch.cat((seq_outputs, features_outputs), dim=1)
    logits = self.classifier(outputs) # calculate losses
    
    loss = None
    if labels is not None:
      loss_fct = torch.nn.CrossEntropyLoss()
      loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
    
    return TokenClassifierOutput(loss=loss, logits=logits, hidden_states=pretrained_outputs.hidden_states ,attentions=pretrained_outputs.attentions)
  
  def save_checkpoint(self, model_name, steps):
    torch.save(self.state_dict(), f'{CHECKPOINTS_PATH}{model_name}_{steps}.pt')
  

In [13]:
if os.path.isfile(f'model/{MODEL_NAME}-custom-{TRAIN_SIZE}') and input('Use pretrain model? (y/n)') == 'y':
  model = torch.load(f'model/{MODEL_NAME}-custom-{TRAIN_SIZE}').to(device)
  print('Using pretrained model')
else:
  model = CustomModel(MODEL_NAME, NUM_LABELS).to(device)
  print('Instantiate new model')

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaModel: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Instantiate new model


In [14]:
optimizer = AdamW(model.parameters(), lr=LR)

num_training_steps = EPOCHS * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=EVAL_SIZE, num_training_steps=num_training_steps
)

In [15]:
f1_metric = evaluate.load("f1")
acu_metric = evaluate.load("accuracy")

def compute_metrics():
    f1_macro = f1_metric.compute(average='macro')['f1']
    acu = acu_metric.compute()['accuracy']
    return {'f1_macro': f1_macro, 'acu': acu}


## Training

In [68]:
progress_bar_train = tqdm(range(num_training_steps))
progress_bar_eval = tqdm(range(EPOCHS * len(eval_dataloader)))

best_f1 = 0
steps = 0
cumulative_loss = 0
for epoch in range(EPOCHS):
  model.train()
  for batch in train_dataloader:
      batch = {k: v.to(device) for k, v in batch.items()}
      outputs = model(**batch)
      loss = outputs.loss
      loss.backward()

      optimizer.step()
      lr_scheduler.step()
      optimizer.zero_grad()
      progress_bar_train.update(1)

      cumulative_loss += loss.item()
      steps += 1

      if steps % CHECKPOINTS_SIZE == 0:
        print(f'step {steps} loss: {cumulative_loss / CHECKPOINTS_SIZE}')
        cumulative_loss = 0
        model.save_checkpoint(MODEL_NAME, steps)

        model.eval()
        for batch in eval_dataloader:
          batch = {k: v.to(device) for k, v in batch.items()}
          with torch.no_grad():
              outputs = model(**batch)

          logits = outputs.logits
          predictions = torch.argmax(logits, dim=-1)
          f1_metric.add_batch(predictions=predictions, references=batch["labels"])
          acu_metric.add_batch(predictions=predictions, references=batch["labels"])
          progress_bar_eval.update(1)
          
        metrics = compute_metrics()
        print(metrics)

        if metrics['f1_macro'] < best_f1 - 0.2:
          print(f'Early stopping at step {steps}, f1 drop {metrics["f1_macro"] - best_f1} from {best_f1} to {metrics["f1_macro"]}')
          break
        best_f1 = max(best_f1, metrics['f1_macro'])

        model.train()

A Jupyter Widget

A Jupyter Widget

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


step 1000 loss: 1.3972753407359124
{'f1_macro': 0.44788661748638825, 'acu': 0.5555}
step 2000 loss: 1.1707748259902
{'f1_macro': 0.5020814179602406, 'acu': 0.5915}
step 3000 loss: 1.1446234025359154
{'f1_macro': 0.5024441902222214, 'acu': 0.584}
step 4000 loss: 1.0156625973284243
{'f1_macro': 0.5149974739328864, 'acu': 0.6035}
step 5000 loss: 0.9935094375014305
{'f1_macro': 0.5281572075712743, 'acu': 0.608}
step 6000 loss: 0.9503588571846485
{'f1_macro': 0.5257820254897689, 'acu': 0.6145}
step 7000 loss: 0.7806139044761657
{'f1_macro': 0.5372428319237448, 'acu': 0.6225}
step 8000 loss: 0.7203718647062779
{'f1_macro': 0.5577827970368987, 'acu': 0.635}
step 9000 loss: 0.6810925497114658
{'f1_macro': 0.5506582006599503, 'acu': 0.6295}


In [16]:
# load checkpoints
model.load_state_dict(torch.load(f'{CHECKPOINTS_PATH}{MODEL_NAME}_{8000}.pt'))
# save
# torch.save(model, f'model/{MODEL_NAME}-custom-{TRAIN_SIZE}x{EPOCHS}')

<All keys matched successfully>

## Evaluation

In [17]:
f1_metric = evaluate.load("f1")
acu_metric = evaluate.load("accuracy")

model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    f1_metric.add_batch(predictions=predictions, references=batch["labels"])
    acu_metric.add_batch(predictions=predictions, references=batch["labels"])

print(compute_metrics())

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/alan/dm/venv/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2322: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


{'f1_macro': 0.5577827970368987, 'acu': 0.635}


## Inference

In [18]:
TEST_BATCH_SIZE = 32

TEST_INPUT_COLUMNS = INPUT_COLUMNS.copy()
if 'label' in TEST_INPUT_COLUMNS:
  TEST_INPUT_COLUMNS.remove('label')

In [19]:
df_test = pd.read_pickle(TEST_DATA_PATH)
df_test.sample(5)

,_score,_crawldate,hashtags,text,joy_ratio,anticipation_ratio,trust_ratio,sadness_ratio,disgust_ratio,fear_ratio,surprise_ratio,anger_ratio
tweet_id,,,,,,,,,,,,
0x30d8de,53,2016-12-25 01:42:11,[],I lost one of my earrings today while at the c...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
0x2e8a3b,940,2015-07-13 02:01:09,[stillhungry],"Asked for a hash brown at German McD’s, I thou...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
0x2bfc9a,558,2017-10-14 14:57:45,"[tired, trauma, mentalhealth]",I survived today. my mind didn’t dissociate. I...,0.200981,0.131773,0.151721,0.252622,0.076753,0.066048,0.020213,0.09989
0x315d94,129,2016-10-09 08:32:01,[],as much as i love travelling nothing feels bet...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
0x260321,732,2015-02-03 15:17:43,[anotherfutureSpartan],@BenBoudro congrats to you and your family. #...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000


In [20]:
test_dataset = Dataset.from_pandas(df_test[TEST_INPUT_COLUMNS])
test_dataset = test_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.remove_columns(["text", 'tweet_id'])
test_dataset.set_format(type='torch')
model.to(device)
test_dataset

A Jupyter Widget

Dataset({
    features: ['joy_ratio', 'anticipation_ratio', 'trust_ratio', 'sadness_ratio', 'disgust_ratio', 'fear_ratio', 'surprise_ratio', 'anger_ratio', 'input_ids', 'attention_mask'],
    num_rows: 411972
})

In [21]:
data_collator = DataCollatorWithPadding(tokenizer)
test_dataloader = DataLoader(test_dataset, batch_size=TEST_BATCH_SIZE, collate_fn=data_collator)

In [22]:
df_result = pd.DataFrame({'id': df_test.index})
df_result['label'] = np.zeros(len(df_result)) - 1

progress_bar_test = tqdm(range(len(test_dataloader)))

idx = 0
model.eval()
for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)

    inc = min(TEST_BATCH_SIZE, len(df_result) - idx)
    df_result.loc[idx:idx+inc-1, 'label'] = predictions.cpu().numpy()
    idx += inc
    progress_bar_test.update(1)

# convert label to emotion
df_result['emotion'] = df_result['label'].map(lambda x: id2emotion[x])

df_result.head(3)

A Jupyter Widget

In [ ]:
saved_path = f'submission/{MODEL_NAME}-custom-{TRAIN_SIZE}x{EPOCHS}.csv'
df_result[['id', 'emotion']].to_csv(saved_path, index=False)
print(saved_path)

submission/cardiffnlp/twitter-roberta-base-sentiment-latest-custom-500000x3.csv


50000